In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U sentence-transformers
!pip install -U qdrant-client
!pip install fastembed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00:00


In [ ]:
PATH_TEST_COLLECTION_DATA="drive/MyDrive/Uni/Master/Masterthesis/Data/test_collection/"
EXPERIMENTS_DATA="drive/MyDrive/Uni/Master/Masterthesis/Experiments/Runs/Data/"

#Load and process data

## Load query data

In [ ]:
import json

# importieren der Pooling results
search_queries_file = PATH_TEST_COLLECTION_DATA + "suchanfragen_testkollektion_final.json"

# JSON direkt einlesen
with open(search_queries_file, "r", encoding="utf-8") as f:
    search_queries = json.load(f)

search_queries[4]

{'id': 'c6',
 'cluster': 'cluster_6',
 'topic': 'Digitales Marketing und Markenkommunikation',
 'single': [{'id': 'c6_1_single',
   'keyword': 'Marketing',
   'variations': [{'id': 'c6_1_single_keyword',
     'query': 'Marketing',
     'type': 'single_keyword'},
    {'id': 'c6_1_single_freetext',
     'query': 'Gesucht wird ein Experte mit Fähigkeiten im Bereich Marketing',
     'type': 'single_free'},
    {'id': 'c6_1_single_keyword_synonym',
     'query': 'Absatzförderung',
     'type': 'single_keyword_synonym'},
    {'id': 'c6_1_single_freetext_synonym',
     'query': 'Gesucht wird ein Experte mit Fähigkeiten im Bereich Absatzförderung',
     'type': 'single_free_synonym'}]},
  {'id': 'c6_2_single',
   'keyword': 'Brand Management',
   'variations': [{'id': 'c6_2_single_keyword',
     'query': 'Brand Management',
     'type': 'single_keyword'},
    {'id': 'c6_2_single_freetext',
     'query': 'Gesucht wird ein Experte mit Fähigkeiten im Bereich Brand Management',
     'type': 'singl

## Process search queries

In [ ]:
def process_query(search_queries_data: list) -> dict:
  """
    This function extracts the different types of search queries
    and put them into a combined list.

    Parameters:
    search_queries_data (list): Raw search queries data

    Returns:
    list[dict]: combines search queries
  """

  queries_data = []

  for search_query in search_queries_data:

    # get single objects
    if search_query["single"]:
        for single_query in search_query["single"]:
          for variation in single_query["variations"]:
            single_query_id = variation["id"]
            single_query_text = variation["query"]
            single_query_type = variation["type"]

            query_object = {
                "id": single_query_id,
                "query": single_query_text,
                "type": single_query_type
            }

            queries_data.append(query_object)

    # get combination objects
    if search_query["combination"]:
        for combination_query in search_query["combination"]:
          for variation in combination_query["variations"]:
            combination_query_id = variation["id"]
            combination_query_text = variation["query"]
            combination_query_type =  variation["type"]

            query_object = {
                "id": combination_query_id,
                "query": combination_query_text,
                "type": combination_query_type
            }

            queries_data.append(query_object)

  return queries_data



In [ ]:
combined_search_queries = process_query(search_queries)
print(len(combined_search_queries))
combined_search_queries

336


[{'id': 'c0_1_single_keyword',
  'query': 'IT-Systemadministration',
  'type': 'single_keyword'},
 {'id': 'c0_1_single_freetext',
  'query': 'Gesucht wird ein Experte mit Fähigkeiten im Bereich IT-Systemadministration',
  'type': 'single_free'},
 {'id': 'c0_1_single_keyword_synonym',
  'query': 'Serveradministration',
  'type': 'single_keyword_synonym'},
 {'id': 'c0_1_single_freetext_synonym',
  'query': 'Gesucht wird ein Experte mit Fähigkeiten im Bereich Serveradministration',
  'type': 'single_free_synonym'},
 {'id': 'c0_2_single_keyword',
  'query': 'Information Technology',
  'type': 'single_keyword'},
 {'id': 'c0_2_single_freetext',
  'query': 'Gesucht wird ein Experte mit Fähigkeiten im Bereich Information Technology',
  'type': 'single_free'},
 {'id': 'c0_2_single_keyword_synonym',
  'query': 'Computing',
  'type': 'single_keyword_synonym'},
 {'id': 'c0_2_single_freetext_synonym',
  'query': 'Gesucht wird ein Experte mit Fähigkeiten im Bereich Computing',
  'type': 'single_free

## Load document data

In [ ]:
import json

# importieren der Pooling results
pooling_file = PATH_TEST_COLLECTION_DATA + "pooling_results_final.json"

# JSON direkt einlesen
with open(pooling_file, "r", encoding="utf-8") as f:
    pooling_results = json.load(f)

pooling_results[0]

{'doc_id': '67251b202f496742be0ea207',
 'doc_raw': '{"branches": [], "companyLocationCity": "Wuppertal", "companyLocationStreet": "Schwelmer Straße 245", "companyLocationZip": 42389.0, "companyName": "Salto Systems GmbH", "companyTypes": [], "companyWebsite": "saltosystems.com/de", "description": "Ich bin ein Enthusiast für elektronische Zutrittskontrolle und Schließsysteme und es begeistert mich, wenn durch integrative Lösungen Probleme gelöst und Prozesse optimiert werden können.", "employeeOfInstitutionNames": [], "firstName": "Bela", "gender": "MALE", "id": "67251b202f496742be0ea207", "jobTitle": "System- und Projektberater", "lastName": "Marahrens", "projectsDescription": "Elektronische Zutrittskontrolle\\nElektronische Schließsysteme\\nBesuchermanagement\\nCloudbasierte Zutrittskontrolle", "skills": [], "title": null, "full_text": "Bela Marahrens ist System- und Projektberater. Er ist Enthusiast für elektronische Zutrittskontrolle und Schließsysteme und überzeugt davon, dass inte

## Process document data

In [ ]:
### create copy without fulltext
import copy
import json

pooling_results_copy = copy.deepcopy(pooling_results)

pooling_results_without_fulltext = []

for obj in pooling_results_copy:
    new_obj = obj.copy()

    if "profile_dict" in new_obj and isinstance(new_obj["profile_dict"], dict):
        # 1) full_text entfernen
        new_profile_dict = {
            k: v for k, v in new_obj["profile_dict"].items()
            if k != "full_text"
        }

        # 2) doc_raw überschreiben
        new_doc_raw = json.dumps(
            new_profile_dict,
            ensure_ascii=False,
            default=str
        )

        new_obj["profile_dict"] = new_profile_dict
        new_obj["doc_raw"] = new_doc_raw

    pooling_results_without_fulltext.append(new_obj)

# Check
print(pooling_results_without_fulltext[0]["profile_dict"].keys())
print(pooling_results_without_fulltext[1]["doc_raw"])
print(len(pooling_results_without_fulltext))


dict_keys(['branches', 'companyLocationCity', 'companyLocationStreet', 'companyLocationZip', 'companyName', 'companyTypes', 'companyWebsite', 'description', 'employeeOfInstitutionNames', 'firstName', 'gender', 'id', 'jobTitle', 'lastName', 'projectsDescription', 'skills', 'title'])
{"branches": ["Technologie"], "companyLocationCity": "Siegen", "companyLocationStreet": "Sonnenstraße 33-35", "companyLocationZip": 57078.0, "companyName": "K-iS Systemhaus GmbH", "companyTypes": ["Dienstleistung"], "companyWebsite": "https://www.k-is.com/", "description": "Als Account Manager mit dem Hintergrund der Heilerziehungspflege bringe ich eine einzigartige Kombination aus Empathie und technischem Know-how mit. \nMeine Kernkompetenz ist es, komplexe Herausforderungen zu erkennen und in einfache Lösungen zu übersetzen.\nFlexibilität und Anpassungsfähigkeit sind die Skills, die ich auch gerne bei Ihnen anwende.\nIch stehe Ihnen als vertrauensvoller Partner zur Seite und unterstütze Sie bei der digital

In [ ]:
## create copy with only fulltext
import copy

pooling_results_copy = copy.deepcopy(pooling_results)

pooling_results_fulltext_only = []

for obj in pooling_results_copy:
    new_obj = obj.copy()

    profile = new_obj.get("profile_dict")

    if isinstance(profile, dict):
        # full_text extrahieren
        full_text = profile.get("full_text")

        # doc_raw entfernen
        new_obj.pop("doc_raw", None)

        # neuen Key setzen
        new_obj["doc_raw_fulltext"] = full_text

    pooling_results_fulltext_only.append(new_obj)

# Check
print(pooling_results_fulltext_only[0].keys())
print(pooling_results_fulltext_only[0])


dict_keys(['doc_id', 'profile_dict', 'doc_raw_fulltext'])
{'doc_id': '67251b202f496742be0ea207', 'profile_dict': {'branches': [], 'companyLocationCity': 'Wuppertal', 'companyLocationStreet': 'Schwelmer Straße 245', 'companyLocationZip': 42389.0, 'companyName': 'Salto Systems GmbH', 'companyTypes': [], 'companyWebsite': 'saltosystems.com/de', 'description': 'Ich bin ein Enthusiast für elektronische Zutrittskontrolle und Schließsysteme und es begeistert mich, wenn durch integrative Lösungen Probleme gelöst und Prozesse optimiert werden können.', 'employeeOfInstitutionNames': [], 'firstName': 'Bela', 'gender': 'MALE', 'id': '67251b202f496742be0ea207', 'jobTitle': 'System- und Projektberater', 'lastName': 'Marahrens', 'projectsDescription': 'Elektronische Zutrittskontrolle\nElektronische Schließsysteme\nBesuchermanagement\nCloudbasierte Zutrittskontrolle', 'skills': [], 'title': None, 'full_text': 'Bela Marahrens ist System- und Projektberater. Er ist Enthusiast für elektronische Zutrittsk

# Create result dataframe

In [ ]:
import hashlib

def make_run_id(search_method, query_id, document_id, document_type, length=12):
    raw = f"{search_method}|{query_id}|{document_id}|{document_type}"
    return hashlib.sha256(raw.encode("utf-8")).hexdigest()[:length]

In [ ]:
columns = [
    "run_id",
    "search_method",
    "model",
    "query_id",
    "query_type",
    "query_text",
    "document_id",
    "document_type",
    "rank",
    "score",
    "duration"
]

In [ ]:
from __future__ import annotations

from typing import Any, Dict, Iterable, List, Optional, Union
import pandas as pd

## Ranking logic überprüfen
def append_search_results(
    df: Optional[pd.DataFrame],
    *,
    search_method: str,
    model: str,
    results_list: Iterable[Dict[str, Any]],
) -> pd.DataFrame:
    """
    Appendet Suchergebnisse (Dokumente) einer einzelnen Query als neue Zeilen an ein DataFrame.

    Erwartet `results` als Iterable von Dicts.
    Rank wird aus der Reihenfolge in `results` erzeugt (rank_start..).

    Returns: neues DataFrame (df erweitert). Wenn df None ist, wird ein neues erstellt.
    """

    if df is None:
        df = pd.DataFrame(columns=columns)
    else:
        # falls df existiert, aber Spalten fehlen (z.B. beim ersten Run)
        missing = [c for c in columns if c not in df.columns]
        if missing:
            for c in missing:
                df[c] = pd.NA
            df = df[columns]

    rows: List[Dict[str, Any]] = []

    # {'query_id': 'c0_1_single_keyword', 'query_text': 'IT-Systemadministration', 'query_type': 'single_keyword', 'document_type': 'fulltext', 'results': [{'score': 0.2971865, 'doc_id': '683d99aea9d017f2c4cf2330'}

    for query_run_results in results_list:
        # doc kann z.B. {"id": "...", "type": "..."} sein oder mehr Felder enthalten
        # hier score extrahieren
        query_id = query_run_results["query_id"]
        query_type = query_run_results["query_type"]
        query_text = query_run_results["query_text"]
        document_type = query_run_results["document_type"]
        duration = query_run_results["duration"]

        for idx, doc in enumerate(query_run_results["results"], start=1):
            document_id = doc["doc_id"]
            score = doc["score"]

            run_id = make_run_id(
                search_method=search_method,
                query_id=query_id,
                document_id=document_id,
                document_type=document_type
            )

            rows.append(
                {
                    "run_id": run_id,
                    "search_method": search_method,
                    "model": model,
                    "query_id": query_id,
                    "query_type": query_type,
                    "query_text": query_text,
                    "document_id": document_id,
                    "document_type": document_type,
                    "rank": idx,
                    "score": score,
                    "duration": duration
                }
            )

    if not rows:
        # keine Ergebnisse -> df unverändert zurück
        return df

    new_df = pd.DataFrame(rows, columns=columns)
    return pd.concat([df, new_df], ignore_index=True)


In [ ]:
from __future__ import annotations

import os
from typing import Optional, Sequence

import pandas as pd


# Expected schema for query result logging (column order is kept stable)
REQUIRED_COLUMNS: Sequence[str] = (
    "run_id",
    "search_method",
    "model",
    "query_id",
    "query_type",
    "query_text",
    "document_id",
    "document_type",
    "rank",
    "score",
    "duration"
)

# Columns used to prevent duplicate entries across runs
DEDUP_KEYS: Sequence[str] = ("run_id",)


def append_query_results_df_to_csv(
    query_results: Optional[pd.DataFrame],
    csv_path: str,
) -> None:
    """
    Append query-level retrieval results to a CSV file while preventing duplicates.

    If the CSV exists, only rows with new (search_method, query_id, document_id)
    combinations are appended. Otherwise, the full DataFrame is written.
    """

    # Nothing to do if input is empty
    if query_results is None or query_results.empty:
        return

    # Ensure required columns are present
    missing_columns = [c for c in REQUIRED_COLUMNS if c not in query_results.columns]
    if missing_columns:
        raise ValueError(
            f"Missing required columns: {missing_columns}. "
            f"Expected columns: {list(REQUIRED_COLUMNS)}"
        )

    # Work on a copy and enforce a stable column order
    df = query_results.loc[:, REQUIRED_COLUMNS].copy()

    # If the CSV does not yet exist, write everything
    if not os.path.exists(csv_path):
        df.to_csv(csv_path, index=False)
        return

    # Load only the key columns from the existing CSV
    existing_keys_df = pd.read_csv(csv_path, usecols=list(DEDUP_KEYS))

    # Build indices for fast comparison of key combinations
    existing_index = pd.MultiIndex.from_frame(existing_keys_df.loc[:, DEDUP_KEYS])
    new_index = pd.MultiIndex.from_frame(df.loc[:, DEDUP_KEYS])

    # Keep only rows that are not yet present in the CSV
    is_new_row = ~new_index.isin(existing_index)
    df_new = df.loc[is_new_row]

    # Exit early if there is nothing new to append
    if df_new.empty:
        return

    # Append new rows without writing the header again
    df_new.to_csv(csv_path, mode="a", header=False, index=False)


In [ ]:
import pandas as pd
from pathlib import Path

search_results_file = Path(EXPERIMENTS_DATA) / "search_results_data.csv"

if search_results_file.exists():
    print("Found")
    search_results = pd.read_csv(search_results_file)
else:
    print("Not Found")
    search_results = pd.DataFrame(
        columns = columns
    )


Found


In [ ]:
search_results

,run_id,search_method,model,query_id,query_type,query_text,document_id,document_type,rank,score,duration
0,a60c18726fa0,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,683d99aea9d017f2c4cf2330,fulltext,1,0.297187,0.672
1,4baa6c7cc291,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,65ba06be3897d6f0e653f0cc,fulltext,2,0.268765,0.672
2,1f19623c7279,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,6682c449680d5b5995b4229d,fulltext,3,0.264832,0.672
3,1f0148601c4c,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,66193c72b5bb2b1c9a963589,fulltext,4,0.246097,0.672
4,4964813cf7da,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,660fe785b5bb2b1c9a9440b3,fulltext,5,0.231186,0.672
...,...,...,...,...,...,...,...,...,...,...,...
184795,decad3b0dac3,cross_encoder,cross-encoder/mmarco-mMiniLMv2-L12-H384-v1,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,65ca9b4561580d8db5ff02b9,structured,46,-5.197807,2.459
184796,78f28b179d7e,cross_encoder,cross-encoder/mmarco-mMiniLMv2-L12-H384-v1,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,6638bc9268131729196ea198,structured,47,-5.210941,2.459
184797,8f024727edd0,cross_encoder,cross-encoder/mmarco-mMiniLMv2-L12-H384-v1,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,6661ab1cd89259daa708b3d3,structured,48,-5.259947,2.459
184798,a0bd354460ed,cross_encoder,cross-encoder/mmarco-mMiniLMv2-L12-H384-v1,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,671f67692f496742be0b1d80,structured,49,-5.276676,2.459


# Runs

In [ ]:
from qdrant_client import models, QdrantClient

In [ ]:
from tqdm import tqdm

In [ ]:
# get qdrant client
client = QdrantClient(
    url="qdrant_url",
    api_key="api_key",
)

In [ ]:
LIMIT = 50

In [ ]:
def get_document_type(collection):
  return collection.split("_", 2)[2]

In [ ]:
import time
from functools import wraps

def measure_latency(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = fn(*args, **kwargs)
        duration = round(time.perf_counter() - start, 3)
        return result, duration
    return wrapper

## BI-Encoder

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
bi_encoder = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
@measure_latency
def query_bi_encoder(search_query: str, collection: str):

      print(search_query)

      rel_docs = client.query_points(
          collection_name=collection,
          query=bi_encoder.encode(search_query).tolist(),
          limit=LIMIT,
      ).points

      return rel_docs


NameError: name 'measure_latency' is not defined

In [ ]:
from re import search
collections = [
    "bi_encoder_fulltext",
    "bi_encoder_structured",
    "bi_encoder_structured_fulltext"
]

for collection in tqdm(collections):

    # get document type from collection name
    document_type = get_document_type(collection)

    queries_results_list = []
    for query in tqdm(combined_search_queries):
      query_id = query["id"]
      query_text = query["query"]
      query_type = query["type"]

      bi_encoder_results, duration = query_bi_encoder(query_text, collection)

      results_list = [
          {"score": r.score, "doc_id": r.payload.get("doc_id")}
          for r in bi_encoder_results
      ]

      queries_results_list.append(
          {
              "query_id": query_id,
              "query_text": query_text,
              "query_type": query_type,
              "document_type": document_type,
              "results": results_list,
              "duration": duration
          }
      )

    search_results = append_search_results(
        df=search_results,
        search_method="bi_encoder",
        model="sentence-transformers/distiluse-base-multilingual-cased-v2",
        results_list=queries_results_list
    )

    append_query_results_df_to_csv(
        query_results=search_results,
        csv_path=search_results_file
    )



  0%|          | 0/336 [00:00<?, ?it/s]

IT-Systemadministration



  1%|          | 2/336 [00:00<02:07,  2.62it/s]

Gesucht wird ein Experte mit Fähigkeiten im Bereich IT-Systemadministration
Serveradministration



  1%|          | 4/336 [00:01<01:21,  4.09it/s]

...


 99%|█████████▉| 334/336 [01:04<00:00,  5.29it/s]

Gesucht wird ein Experte mit Fähigkeiten in den Bereichen Business Development und Business Process Analysis
Geschäftsentwicklung, Geschäftsprozessanalyse



100%|██████████| 336/336 [01:04<00:00,  5.22it/s]


Gesucht wird ein Experte mit Fähigkeiten in den Bereichen Geschäftsentwicklung und Geschäftsprozessanalyse


100%|██████████| 3/3 [03:13<00:00, 64.58s/it]


In [ ]:
search_results

,run_id,search_method,model,query_id,query_type,query_text,document_id,document_type,rank,score,duration
0,a60c18726fa0,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,683d99aea9d017f2c4cf2330,fulltext,1,0.297187,0.672
1,4baa6c7cc291,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,65ba06be3897d6f0e653f0cc,fulltext,2,0.268765,0.672
2,1f19623c7279,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,6682c449680d5b5995b4229d,fulltext,3,0.264832,0.672
3,1f0148601c4c,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,66193c72b5bb2b1c9a963589,fulltext,4,0.246097,0.672
4,4964813cf7da,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,660fe785b5bb2b1c9a9440b3,fulltext,5,0.231186,0.672
...,...,...,...,...,...,...,...,...,...,...,...
50395,76ad64936ab6,bi_encoder,sentence-transformers/distiluse-base-multiling...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,67d44084b5308acb98802c45,structured_fulltext,46,0.114559,0.178
50396,b274b2737ecc,bi_encoder,sentence-transformers/distiluse-base-multiling...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,6787e53d5d12164fde53eb7a,structured_fulltext,47,0.111945,0.178
50397,8caaafc7fcaf,bi_encoder,sentence-transformers/distiluse-base-multiling...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,671f67082f496742be0b1d07,structured_fulltext,48,0.111676,0.178
50398,94886549dd06,bi_encoder,sentence-transformers/distiluse-base-multiling...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,65b817053897d6f0e6537dce,structured_fulltext,49,0.111364,0.178


## Late Interaction

In [ ]:
from fastembed import LateInteractionTextEmbedding

late_model = LateInteractionTextEmbedding("jinaai/jina-colbert-v2")
print(late_model.embedding_size)

128


In [ ]:
@measure_latency
def query_late_interaction(search_query: str, collection: str):

      print(search_query)

      print(collection)

      query_embedding = next(late_model.query_embed([search_query])).tolist()

      rel_docs = client.query_points(
          collection_name=collection,
          query=query_embedding,
          using="late_interaction",
          limit=LIMIT,
      ).points

      return rel_docs


In [ ]:
collections = [ # hier anpassen für andere Methode
  "late_interaction_fulltext",
  "late_interaction_structured",
  "late_interaction_structured_fulltext"
]

for collection in tqdm(collections):

    # get document type from collection name
    document_type = get_document_type(collection)

    queries_results_list = []
    for query in tqdm(combined_search_queries):
      query_id = query["id"]
      query_text = query["query"]
      query_type = query["type"]

      late_interaction_results, duration = query_late_interaction(query_text, collection) # hier anpassen für andere Methode

      results_list = [
          {"score": r.score, "doc_id": r.payload.get("doc_id")}
          for r in late_interaction_results
      ]

      queries_results_list.append(
          {
              "query_id": query_id,
              "query_text": query_text,
              "query_type": query_type,
              "document_type": document_type,
              "results": results_list,
              "duration": duration
          }
      )

    search_results = append_search_results(
        df=search_results,
        search_method="late_interaction",
        model="jinaai/jina-colbert-v2",
        results_list=queries_results_list
    )

    append_query_results_df_to_csv(
        query_results=search_results,
        csv_path=search_results_file
    )

  0%|          | 0/336 [00:00<?, ?it/s]

IT-Systemadministration
late_interaction_fulltext



  0%|          | 1/336 [00:01<08:27,  1.51s/it]

Gesucht wird ein Experte mit Fähigkeiten im Bereich IT-Systemadministration
late_interaction_fulltext



  1%|          | 2/336 [00:02<06:13,  1.12s/it]

...


 99%|█████████▉| 334/336 [06:33<00:02,  1.09s/it]

Geschäftsentwicklung, Geschäftsprozessanalyse
late_interaction_structured_fulltext



100%|█████████▉| 335/336 [06:34<00:01,  1.09s/it]

Gesucht wird ein Experte mit Fähigkeiten in den Bereichen Geschäftsentwicklung und Geschäftsprozessanalyse
late_interaction_structured_fulltext



100%|██████████| 3/3 [17:43<00:00, 354.60s/it]


In [ ]:
search_results

,run_id,search_method,model,query_id,query_type,query_text,document_id,document_type,rank,score,duration
0,a60c18726fa0,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,683d99aea9d017f2c4cf2330,fulltext,1,0.297187,0.672
1,4baa6c7cc291,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,65ba06be3897d6f0e653f0cc,fulltext,2,0.268765,0.672
2,1f19623c7279,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,6682c449680d5b5995b4229d,fulltext,3,0.264832,0.672
3,1f0148601c4c,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,66193c72b5bb2b1c9a963589,fulltext,4,0.246097,0.672
4,4964813cf7da,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,660fe785b5bb2b1c9a9440b3,fulltext,5,0.231186,0.672
...,...,...,...,...,...,...,...,...,...,...,...
100795,acd1ff1a9a24,late_interaction,jinaai/jina-colbert-v2,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,66d5c842528e751260e5401d,structured_fulltext,46,18.270103,1.078
100796,4e743a6cd10a,late_interaction,jinaai/jina-colbert-v2,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,673ae42551c382bea6145daf,structured_fulltext,47,18.268606,1.078
100797,4767ae53e4d2,late_interaction,jinaai/jina-colbert-v2,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,686e5c6db1e17e69a9cb8b53,structured_fulltext,48,18.267958,1.078
100798,622469796eb7,late_interaction,jinaai/jina-colbert-v2,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,66eabc0d41f659468bdd8da8,structured_fulltext,49,18.266151,1.078


## Neural Sparse

In [ ]:
from sentence_transformers.sparse_encoder import SparseEncoder

In [ ]:
# Download from the 🤗 Hub
sparse_model = SparseEncoder("opensearch-project/opensearch-neural-sparse-encoding-multilingual-v1")

modules.json:   0%|          | 0.00/108 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

router_config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/872 [00:00<?, ?B/s]

./model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/106 [00:00<?, ?B/s]

In [ ]:
def encode_sparse_for_qdrant(text: str) -> models.SparseVector:

    tensor = sparse_model.encode_document(text)      # opensearch sparse encoder
    tensor = tensor.coalesce()

    idx = tensor.indices()   # Shape: [1, nnz]
    vals = tensor.values()   # Shape: [nnz]

    token_indices = idx[0].tolist()
    token_values = vals.tolist()

    return models.SparseVector(
        indices=token_indices,
        values=token_values,
    )

In [ ]:
@measure_latency
def query_neural_sparse(search_query: str, collection: str):

      print(search_query)

      print(collection)

      rel_docs = client.query_points(
        collection_name=collection,
        query=encode_sparse_for_qdrant(search_query),
        using="neural_sparse_vector",
        limit=LIMIT,
      ).points

      return rel_docs

In [ ]:
collections = [ # hier anpassen für andere Methode
  "neural_sparse_fulltext",
  "neural_sparse_structured",
  "neural_sparse_structured_fulltext"
]

for collection in tqdm(collections):

    # get document type from collection name
    document_type = get_document_type(collection)

    queries_results_list = []
    for query in tqdm(combined_search_queries):
      query_id = query["id"]
      query_text = query["query"]
      query_type = query["type"]

      neural_sparse_results, duration = query_neural_sparse(query_text, collection) # hier anpassen für andere Methode

      results_list = [
          {"score": r.score, "doc_id": r.payload.get("doc_id")}
          for r in neural_sparse_results
      ]

      queries_results_list.append(
          {
              "query_id": query_id,
              "query_text": query_text,
              "query_type": query_type,
              "document_type": document_type,
              "results": results_list,
              "duration": duration
          }
      )

    search_results = append_search_results(
        df=search_results,
        search_method="neural_sparse",
        model="opensearch-project/opensearch-neural-sparse-encoding-multilingual-v1",
        results_list=queries_results_list
    )

    append_query_results_df_to_csv(
        query_results=search_results,
        csv_path=search_results_file
    )


  0%|          | 0/336 [00:00<?, ?it/s]

IT-Systemadministration
neural_sparse_fulltext



  0%|          | 1/336 [00:01<06:41,  1.20s/it]

Gesucht wird ein Experte mit Fähigkeiten im Bereich IT-Systemadministration
neural_sparse_fulltext



  1%|          | 3/336 [00:01<02:22,  2.34it/s]

...


 99%|█████████▉| 333/336 [01:06<00:00,  5.01it/s]

Business Development, Business Process Analysis
neural_sparse_structured_fulltext
Gesucht wird ein Experte mit Fähigkeiten in den Bereichen Business Development und Business Process Analysis
neural_sparse_structured_fulltext



100%|█████████▉| 335/336 [01:07<00:00,  5.02it/s]

Geschäftsentwicklung, Geschäftsprozessanalyse
neural_sparse_structured_fulltext
Gesucht wird ein Experte mit Fähigkeiten in den Bereichen Geschäftsentwicklung und Geschäftsprozessanalyse
neural_sparse_structured_fulltext



100%|██████████| 3/3 [03:26<00:00, 68.96s/it]


In [ ]:
search_results

,run_id,search_method,model,query_id,query_type,query_text,document_id,document_type,rank,score,duration
0,a60c18726fa0,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,683d99aea9d017f2c4cf2330,fulltext,1,0.297187,0.516
1,4baa6c7cc291,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,65ba06be3897d6f0e653f0cc,fulltext,2,0.268765,0.516
2,1f19623c7279,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,6682c449680d5b5995b4229d,fulltext,3,0.264832,0.516
3,1f0148601c4c,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,66193c72b5bb2b1c9a963589,fulltext,4,0.246097,0.516
4,4964813cf7da,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,660fe785b5bb2b1c9a9440b3,fulltext,5,0.231186,0.516
...,...,...,...,...,...,...,...,...,...,...,...
151195,fd8005c1a10b,neural_sparse,opensearch-project/opensearch-neural-sparse-en...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,67d44084b5308acb98802c45,structured_fulltext,46,8.360296,0.577
151196,a5a35de1bcca,neural_sparse,opensearch-project/opensearch-neural-sparse-en...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,65fae369763465cfa7e2b79f,structured_fulltext,47,8.234073,0.577
151197,e753818a8c83,neural_sparse,opensearch-project/opensearch-neural-sparse-en...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,65b76b7c3897d6f0e6533d5f,structured_fulltext,48,8.195609,0.577
151198,05e28339af41,neural_sparse,opensearch-project/opensearch-neural-sparse-en...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,65e6da95763465cfa7de9283,structured_fulltext,49,8.131554,0.577


In [ ]:
search_results

,run_id,search_method,model,query_id,query_type,query_text,document_id,document_type,rank,score,duration
0,a60c18726fa0,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,683d99aea9d017f2c4cf2330,fulltext,1,0.297187,0.672
1,4baa6c7cc291,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,65ba06be3897d6f0e653f0cc,fulltext,2,0.268765,0.672
2,1f19623c7279,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,6682c449680d5b5995b4229d,fulltext,3,0.264832,0.672
3,1f0148601c4c,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,66193c72b5bb2b1c9a963589,fulltext,4,0.246097,0.672
4,4964813cf7da,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,660fe785b5bb2b1c9a9440b3,fulltext,5,0.231186,0.672
...,...,...,...,...,...,...,...,...,...,...,...
151195,fd8005c1a10b,neural_sparse,opensearch-project/opensearch-neural-sparse-en...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,67d44084b5308acb98802c45,structured_fulltext,46,8.360318,0.195
151196,a5a35de1bcca,neural_sparse,opensearch-project/opensearch-neural-sparse-en...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,65fae369763465cfa7e2b79f,structured_fulltext,47,8.234089,0.195
151197,e753818a8c83,neural_sparse,opensearch-project/opensearch-neural-sparse-en...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,65b76b7c3897d6f0e6533d5f,structured_fulltext,48,8.195627,0.195
151198,05e28339af41,neural_sparse,opensearch-project/opensearch-neural-sparse-en...,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,65e6da95763465cfa7de9283,structured_fulltext,49,8.131570,0.195


## Cross Encoder

In [ ]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder("cross-encoder/mmarco-mMiniLMv2-L12-H384-v1")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/891 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
@measure_latency
def query_cross_encoder(query: str, documents: list):

    if not documents:
        return []

    # Look for avaiable key
    text_key = "doc_raw_fulltext" if "doc_raw_fulltext" in documents[0] else "doc_raw"

    # Test if key is avaiable
    missing = [i for i, d in enumerate(documents) if text_key not in d]
    if missing:
        raise KeyError(f"{text_key} fehlt in docs an Indizes: {missing[:10]} (total={len(missing)})")

    print(text_key)

    #[print(doc[text_key]) for doc in documents[:1]]

    pairs = [(query, doc[text_key]) for doc in documents]
    scores = cross_encoder.predict(pairs, batch_size=32)

    top_idx = sorted(
        range(len(scores)),
        key=lambda i: scores[i],
        reverse=True
    )[:LIMIT]

    rel_docs = [
        {
            "score": float(scores[i]),
            "doc_id": documents[i]["doc_id"]
        }
        for i in top_idx
    ]

    return rel_docs


In [ ]:
collections = [
    {
        "documents": pooling_results_fulltext_only,
        "document_type": "fulltext"
    },
    {
        "documents": pooling_results_without_fulltext,
        "document_type": "structured"
    },
    {
        "documents": pooling_results,
        "document_type": "structured_fulltext"
    },

]

for collection in tqdm(collections[2:]):

    # get document type from collection name
    document_type = collection["document_type"]

    print(document_type)

    queries_results_list = []
    for query in tqdm(combined_search_queries):
      query_id = query["id"]
      query_text = query["query"]
      query_type = query["type"]

      cross_encoder_results, duration = query_cross_encoder(query_text, collection["documents"]) # hier anpassen für andere Methode

      results_list = [
          {"score": r.get("score"), "doc_id": r.get("doc_id")}
          for r in cross_encoder_results
      ]

      queries_results_list.append(
          {
              "query_id": query_id,
              "query_text": query_text,
              "query_type": query_type,
              "document_type": document_type,
              "results": results_list,
              "duration": duration
          }
      )

    search_results = append_search_results(
        df=search_results,
        search_method="cross_encoder",
        model="cross-encoder/mmarco-mMiniLMv2-L12-H384-v1",
        results_list=queries_results_list
    )

    append_query_results_df_to_csv(
        query_results=search_results,
        csv_path=search_results_file
    )

  0%|          | 0/1 [00:00<?, ?it/s]

structured_fulltext



  0%|          | 0/336 [00:00<?, ?it/s]

doc_raw



  0%|          | 1/336 [00:03<19:05,  3.42s/it]

...


 99%|█████████▉| 334/336 [14:31<00:05,  2.64s/it]

doc_raw



100%|█████████▉| 335/336 [14:34<00:02,  2.63s/it]

doc_raw



100%|██████████| 1/1 [14:38<00:00, 878.40s/it]


In [ ]:
search_results

,run_id,search_method,model,query_id,query_type,query_text,document_id,document_type,rank,score,duration
0,a60c18726fa0,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,683d99aea9d017f2c4cf2330,fulltext,1,0.297187,0.672
1,4baa6c7cc291,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,65ba06be3897d6f0e653f0cc,fulltext,2,0.268765,0.672
2,1f19623c7279,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,6682c449680d5b5995b4229d,fulltext,3,0.264832,0.672
3,1f0148601c4c,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,66193c72b5bb2b1c9a963589,fulltext,4,0.246097,0.672
4,4964813cf7da,bi_encoder,sentence-transformers/distiluse-base-multiling...,c0_1_single_keyword,single_keyword,IT-Systemadministration,660fe785b5bb2b1c9a9440b3,fulltext,5,0.231186,0.672
...,...,...,...,...,...,...,...,...,...,...,...
201595,62fe49b7eaef,cross_encoder,cross-encoder/mmarco-mMiniLMv2-L12-H384-v1,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,6645e131f48ebd8f3f48df57,structured_fulltext,46,-1.401143,2.625
201596,8c5b7f94a33a,cross_encoder,cross-encoder/mmarco-mMiniLMv2-L12-H384-v1,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,65fae668763465cfa7e2b971,structured_fulltext,47,-1.429569,2.625
201597,ebf170f63e04,cross_encoder,cross-encoder/mmarco-mMiniLMv2-L12-H384-v1,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,66eabc0d41f659468bdd8da8,structured_fulltext,48,-1.476483,2.625
201598,bdc5a6737787,cross_encoder,cross-encoder/mmarco-mMiniLMv2-L12-H384-v1,c13_5_combination_freetext_synonym,combination_free_synonym,Gesucht wird ein Experte mit Fähigkeiten in de...,65f937c1763465cfa7e252ad,structured_fulltext,49,-1.532228,2.625
